In [1]:
import cv2
import os
import numpy as np

# Definir la ruta de la carpeta de imágenes y la carpeta de salida para las imágenes aumentadas
ruta_carpeta = '../Nuevas_fotos'
ruta_salida = '../Caras_aumentadas'

# Crear la carpeta de salida si no existe
if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Funciones para diferentes tipos de aumentaciones
def rotar_imagen(imagen, angulo):
    (h, w) = imagen.shape[:2]
    centro = (w // 2, h // 2)
    matriz_rotacion = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    imagen_rotada = cv2.warpAffine(imagen, matriz_rotacion, (w, h))
    return imagen_rotada

def desplazar_imagen(imagen, x, y):
    matriz_desplazamiento = np.float32([[1, 0, x], [0, 1, y]])
    imagen_desplazada = cv2.warpAffine(imagen, matriz_desplazamiento, (imagen.shape[1], imagen.shape[0]))
    return imagen_desplazada

def escalar_imagen(imagen, escala):
    h, w = imagen.shape[:2]
    imagen_escalada = cv2.resize(imagen, (int(w * escala), int(h * escala)))
    return imagen_escalada

def aplicar_cutout(imagen, tamano_recorte):
    """Aplica un recorte (cutout) negro a la imagen en una ubicación aleatoria."""
    h, w = imagen.shape[:2]
    x_inicio = np.random.randint(0, w - tamano_recorte)
    y_inicio = np.random.randint(0, h - tamano_recorte)
    imagen_cutout = imagen.copy()
    imagen_cutout[y_inicio:y_inicio+tamano_recorte, x_inicio:x_inicio+tamano_recorte] = 0
    return imagen_cutout

# Aplicar aumentaciones a todas las imágenes de la carpeta
for persona in os.listdir(ruta_carpeta):
    ruta_persona = os.path.join(ruta_carpeta, persona)
    if os.path.isdir(ruta_persona):
        ruta_salida_persona = os.path.join(ruta_salida, persona)
        if not os.path.exists(ruta_salida_persona):
            os.makedirs(ruta_salida_persona)
        
        for archivo in os.listdir(ruta_persona):
            ruta_imagen = os.path.join(ruta_persona, archivo)
            imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)  # Leer la imagen en escala de grises
            
            if imagen is not None:
                # Guardar la imagen original (opcional)
                cv2.imwrite(os.path.join(ruta_salida_persona, archivo), imagen)
                
                # Aplicar rotaciones
                for angulo in [15, -15, 30, -30]:
                    imagen_rotada = rotar_imagen(imagen, angulo)
                    nombre_archivo = f"{archivo.split('.')[0]}_rotada_{angulo}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_rotada)
                
                # Aplicar desplazamientos
                for desplazamiento in [(5, 5), (-5, -5), (10, 0), (0, 10)]:
                    imagen_desplazada = desplazar_imagen(imagen, desplazamiento[0], desplazamiento[1])
                    nombre_archivo = f"{archivo.split('.')[0]}_desplazada_{desplazamiento[0]}_{desplazamiento[1]}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_desplazada)
                
                # Aplicar escalado
                for escala in [1.1, 0.9]:
                    imagen_escalada = escalar_imagen(imagen, escala)
                    # Redimensionar de nuevo a 30x30 si el escalado cambió el tamaño
                    imagen_escalada = cv2.resize(imagen_escalada, (30, 30))
                    nombre_archivo = f"{archivo.split('.')[0]}_escalada_{escala}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_escalada)

                # Aplicar cutout
                for tamano in [5, 10, 15]:  # Puedes ajustar el tamaño del recorte según lo desees
                    imagen_cutout = aplicar_cutout(imagen, tamano)
                    nombre_archivo = f"{archivo.split('.')[0]}_cutout_{tamano}.jpg"
                    cv2.imwrite(os.path.join(ruta_salida_persona, nombre_archivo), imagen_cutout)

print("Aumentación de imágenes completada con todas las técnicas.")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '../Nuevas_fotos'